<div style="color:#006666; padding:0px 10px; border-radius:5px; font-size:18px;"><h1 style='margin:10px 5px'>Transforming Data</h1>
</div>

© Copyright Machine Learning Plus

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Assign, Map and Applymap</h2>
</div>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets/Tips100.csv')
df.head()

__assign()__

__Task:__ 

How to create a new column called `tip_perc` showing tips as a percentage of the total bill, and another column `tip_per_person`.

You can use the `assign` method to create multiple columns in a dataframe in one shot. 

It accepts a lambda function, which takes the _entire dataframe_ as an argument.

In [ ]:
df = df.assign(tip_perc = lambda x: round(x['tip']/x['total_bill'], 4),
              tip_per_person = lambda x: round(x['tip']/x['size'], 4))
df.head()

__map()__

__Task:__ 

Encode the `day` variable to show the number of the week day, rather than an abbreviation.

The map function is used to transform every item in a column based on a `lambda function logic` or a `dict` lookup.

Note: `DataFrame` does not have a `map` method. Only a `Series` does.

__Example 1:__ dict based map.

Number code the days of the week based on the dict `weekdays`.

In [ ]:
weekdays = {'Mon':1, 'Tue': 2, 'Wed': 3, 'Thur': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7}

In [ ]:
df['day'] = df['day'].map(weekdays)
df.head()

__Example 2:__ Lambda function based map

__Task__: Mask the CC number and show only the last 4 digits.

In [ ]:
df['CC Number'].map(lambda x: "x"+str(x)[-4:])

If length != 16, show 'xxxx'

In [ ]:
df['CC Number'] = df['CC Number'].map(lambda x: "x"+str(x)[-4:] if len(str(x))==16 else 'xxxx')
df.head()

__applymap()__

`applymap()` is the `DataFrame` version of a `map`. It applies a logic for transformation to every element of a DataFrame.

__Task:__ For all alphabetic cells, replace value with the number of characters in the cell.

In [ ]:
df.applymap(lambda x: str(x).upper()[0] if str(x).isalpha() else x)

The `Payer Name` is untouched because of the space character. So, remove space and try again.

In [ ]:
df.applymap(lambda x: len(str(x)) if str(x).replace(" ", "").isalpha() else x)

### Mini Challenge

Extract the day of the payment from `Payment_ID` column.

__Input Code:__

```python
import pandas as pd
df = pd.read_csv('Datasets/Tips100.csv')
df.head()
```

__Desired Output:__
```python
#> 0     Sat
#> 1    Thur
#> 2    Thur
#> 3    Thur
#> 4    Thur
#> ...
```

Code Link: https://git.io/JskgE

In [ ]:
# Solution
import pandas as pd
df = pd.read_csv('Datasets/Tips100.csv')
df.head()

In [ ]:
def remove_digits(s):
    return ''.join([i for i in s if not i.isdigit()])

In [ ]:
df['Payment_Day'] = df['Payment ID'].map(remove_digits)
df['Payment_Day'][:5]

[Reference Link](https://stackoverflow.com/questions/12851791/removing-numbers-from-string)

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Apply a function Row wise or Column Wise</h2>
</div>

The `apply()` function is one of the challenging functions to master in pandas, let's understand clearly with examples.

The `apply()` method allows you to apply a function (pre-existing or user defined) on every row or every column of a dataframe. It can be applied column-wise or row-wise. Correspondingly, the entire row or column is passed as argument to the function being applied.

Applies to series as well.

__Dataset:__ Hungary Chickenpox

Number of weekly chickenpox cases in various hungary regions.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.head()

__Example 1:__ 

Create a new column that contains the the highest number of Chickenpox cases for any region.

In [ ]:
# Max value in each row
max_cases = df.drop(columns="Date").apply(np.max, axis="columns")
max_cases

__With For Loop__

In [ ]:
# With For Loop
max_values = []
for i, row in enumerate(df.iterrows()):
        max_values.append(np.max(row[1][1:]))

In [ ]:
# Check
max_values[:5]    

__Example 2:__ 

What is the median number of Chickepox cases for every region (column)

In [ ]:
df.drop(columns="Date").apply(np.median, axis="rows")

### Challenge

__1__. Create a column containing the difference between the highest and the lowest number of chickenpox cases.

In [ ]:
df.drop(columns="Date").apply(lambda x: np.max(x) - np.min(x), axis="columns")

__2:__ What is the date each of the region received the highest and lowest number of cases.

In [ ]:
highest_index = df.drop(columns="Date").apply(np.argmax, axis='rows')
lowest_index = df.drop(columns="Date").apply(np.argmin, axis='rows')
print(highest_index)
print("\n")
print(lowest_index)

Check the maximum values. Then check if that value is present in the index 158 for BUDAPEST.

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')

In [ ]:
# Check the maximum values.
df.drop(columns="Date").apply(np.max, axis="rows")

In [ ]:
# At position 158, Budapest should have 479
df.iloc[158, :]

So Good!

In [ ]:
# Rename Index and Concatenate.
highest = df.Date.iloc[highest_index].reset_index().rename(index=lambda x: df.columns[1:][x])
lowest = df.Date.iloc[lowest_index].reset_index().rename(index=lambda x: df.columns[1:][x])
pd.concat([highest, lowest], axis=1)

__3__. Home work:  Which is the region that received the lowest and the highest number of cases in each region?

In [ ]:
# Home work

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Scaling and Standardization</h2>
</div>

__Task:__ 

Bring all the columns in the dataframe to a similar scale so that when a new observation comes, you can clearly understand the highness/lowness of the data irrescpective of the scale.

__Scaling methods:__

__1. Z Score scaling:__  (X - mean)/Std

__2. Min Max Scaling:__  (X - min)/(max - min)

__3. Maximum Absolute Scaling:__ x / max

__4. Robust Scaling:__ x - median / (Q3 - Q1) - Preferred in presence of outliers in data.



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.head()

In [ ]:
df.describe()

__Z Score Scaling__


In [ ]:
def z_score(x):
    return (x - np.mean(x))/np.std(x)

In [ ]:
f = df.drop('Date', axis='columns').apply(z_score, axis='rows')
f

In [ ]:
f.describe()

### Challenge: 

__Task:__ 

1. On the following series, implement:


- Robust Scaling
- Min-Max Scaling


```python
x = pd.Series([1,2,3,4,5,6,7,30])
```

[Ref](https://towardsdatascience.com/data-normalization-with-pandas-and-scikit-learn-7c1cc6ed6475) 

In [ ]:
x = pd.Series([1,2,3,4,5,6,7,30])

In [ ]:
def robust_scaling(x):
    return (x - np.median(x))/(x.quantile(q=.75) - x.quantile(q=.25))

def min_max_scaling(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))

__Robust Scaling__

In [ ]:
robust_scaling(x)

__Min Max Scaling__

In [ ]:
min_max_scaling(x)

__Additional Practice:__ Implement Min-Max Scaling and Maximum absolute scaling.

<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Make Index as a Dataframe Column</h2>
</div>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.head()

,Date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,03/01/2005,168,79,30,173,169,42,136,120,162,...,130,57,2,178,66,64,11,29,87,68
1,10/01/2005,157,60,30,92,200,53,51,70,84,...,80,50,29,141,48,29,58,53,68,26
2,17/01/2005,96,44,31,86,93,30,93,84,191,...,64,46,4,157,33,33,24,18,62,44
3,24/01/2005,163,49,43,126,46,39,52,114,107,...,63,54,14,107,66,50,25,21,43,31
4,31/01/2005,122,78,53,87,103,34,95,131,172,...,61,49,11,124,63,56,7,47,85,60


__Task:__ Convert the date column to an index.

In [ ]:
df.set_index('Date', inplace=True)
df.head()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
Date,,,,,,,,,,,,,,,,,,,,
03/01/2005,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
10/01/2005,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
17/01/2005,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
24/01/2005,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
31/01/2005,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60


__Task:__ Convert the date index back to column

__Method 1:__ Create a new column called `Date`. The new column will be created at the end.

In [ ]:
df['Date'] = df.index
df.head()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,...,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA,Date
Date,,,,,,,,,,,,,,,,,,,,,
03/01/2005,168,79,30,173,169,42,136,120,162,36,...,57,2,178,66,64,11,29,87,68,03/01/2005
10/01/2005,157,60,30,92,200,53,51,70,84,28,...,50,29,141,48,29,58,53,68,26,10/01/2005
17/01/2005,96,44,31,86,93,30,93,84,191,51,...,46,4,157,33,33,24,18,62,44,17/01/2005
24/01/2005,163,49,43,126,46,39,52,114,107,42,...,54,14,107,66,50,25,21,43,31,24/01/2005
31/01/2005,122,78,53,87,103,34,95,131,172,40,...,49,11,124,63,56,7,47,85,60,31/01/2005


__To insert in the first position:__

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.set_index('Date', inplace=True)
df.head()

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
Date,,,,,,,,,,,,,,,,,,,,
03/01/2005,168,79,30,173,169,42,136,120,162,36,130,57,2,178,66,64,11,29,87,68
10/01/2005,157,60,30,92,200,53,51,70,84,28,80,50,29,141,48,29,58,53,68,26
17/01/2005,96,44,31,86,93,30,93,84,191,51,64,46,4,157,33,33,24,18,62,44
24/01/2005,163,49,43,126,46,39,52,114,107,42,63,54,14,107,66,50,25,21,43,31
31/01/2005,122,78,53,87,103,34,95,131,172,40,61,49,11,124,63,56,7,47,85,60


In [ ]:
df.insert(0, "date", df.index)

In [ ]:
df.head()

,date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
Date,,,,,,,,,,,,,,,,,,,,,
03/01/2005,03/01/2005,168,79,30,173,169,42,136,120,162,...,130,57,2,178,66,64,11,29,87,68
10/01/2005,10/01/2005,157,60,30,92,200,53,51,70,84,...,80,50,29,141,48,29,58,53,68,26
17/01/2005,17/01/2005,96,44,31,86,93,30,93,84,191,...,64,46,4,157,33,33,24,18,62,44
24/01/2005,24/01/2005,163,49,43,126,46,39,52,114,107,...,63,54,14,107,66,50,25,21,43,31
31/01/2005,31/01/2005,122,78,53,87,103,34,95,131,172,...,61,49,11,124,63,56,7,47,85,60


__Method 2:__ Use `reset_index()` method.

In [ ]:
df.reset_index()

,Date,date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,03/01/2005,03/01/2005,168,79,30,173,169,42,136,120,...,130,57,2,178,66,64,11,29,87,68
1,10/01/2005,10/01/2005,157,60,30,92,200,53,51,70,...,80,50,29,141,48,29,58,53,68,26
2,17/01/2005,17/01/2005,96,44,31,86,93,30,93,84,...,64,46,4,157,33,33,24,18,62,44
3,24/01/2005,24/01/2005,163,49,43,126,46,39,52,114,...,63,54,14,107,66,50,25,21,43,31
4,31/01/2005,31/01/2005,122,78,53,87,103,34,95,131,...,61,49,11,124,63,56,7,47,85,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,01/12/2014,01/12/2014,95,12,41,6,39,0,16,15,...,56,7,13,122,4,23,4,11,110,10
518,08/12/2014,08/12/2014,43,39,31,10,34,3,2,30,...,34,20,18,70,36,5,23,22,63,9
519,15/12/2014,15/12/2014,35,7,15,0,0,0,7,7,...,30,36,4,72,5,21,14,0,17,10
520,22/12/2014,22/12/2014,30,23,8,0,11,4,1,9,...,27,17,21,12,5,17,1,1,83,2


<div class="alert alert-info" style="background-color:#006666; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Discretization and Binning</h2>
</div>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.head()

,Date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,03/01/2005,168,79,30,173,169,42,136,120,162,...,130,57,2,178,66,64,11,29,87,68
1,10/01/2005,157,60,30,92,200,53,51,70,84,...,80,50,29,141,48,29,58,53,68,26
2,17/01/2005,96,44,31,86,93,30,93,84,191,...,64,46,4,157,33,33,24,18,62,44
3,24/01/2005,163,49,43,126,46,39,52,114,107,...,63,54,14,107,66,50,25,21,43,31
4,31/01/2005,122,78,53,87,103,34,95,131,172,...,61,49,11,124,63,56,7,47,85,60


__Task__: 

Bin a the BUDAPEST variable into four equal sized bins, so you know the classification of whether the number of cases in the state is 'low' / 'medium' / 'high' / 'very high'.

You can use `pd.qcut` for this. Pandas will do the math behind the scenes to determine how to divide the data set into these 4 groups.

In [ ]:
# qcut bins each item into the respective bucket
pd.qcut(df['BUDAPEST'], q=4).head()

0    (149.0, 479.0]
1    (149.0, 479.0]
2     (93.0, 149.0]
3    (149.0, 479.0]
4     (93.0, 149.0]
Name: BUDAPEST, dtype: category
Categories (4, interval[float64]): [(-0.001, 34.25] < (34.25, 93.0] < (93.0, 149.0] < (149.0, 479.0]]

(149.0, 479.0] -> means 149.0 is not included whereas 479.0 is included.

Convert the numbers to labels

In [ ]:
# qcut with labels
df['BUDAPEST_Q'] = pd.qcut(df['BUDAPEST'], q=4, labels=['LOW', 'MEDIUM', 'HIGH', 'VERY HIGH'])
df['BUDAPEST_Q'].head()

0    VERY HIGH
1    VERY HIGH
2         HIGH
3    VERY HIGH
4         HIGH
Name: BUDAPEST_Q, dtype: category
Categories (4, object): ['LOW' < 'MEDIUM' < 'HIGH' < 'VERY HIGH']

__New Task:__ 

Bin `BUDAPEST` based on predetermined buckets: [10, 50, 100]

You need to make sure all the possible cases will be covered by the specified bin, else, expand your bin ranges.

In [ ]:
# cut at predetermined edges
pd.cut(df['BUDAPEST'], bins=[0, 10, 50, 100, 1000])

0      (100, 1000]
1      (100, 1000]
2        (50, 100]
3      (100, 1000]
4      (100, 1000]
          ...     
517      (50, 100]
518       (10, 50]
519       (10, 50]
520       (10, 50]
521    (100, 1000]
Name: BUDAPEST, Length: 522, dtype: category
Categories (4, interval[int64]): [(0, 10] < (10, 50] < (50, 100] < (100, 1000]]

In [ ]:
# cut with labels
pd.cut(df['BUDAPEST'], bins=[0, 10, 50, 100, 1000], labels=[1,2,3,4])

0      4
1      4
2      3
3      4
4      4
      ..
517    3
518    2
519    2
520    2
521    4
Name: BUDAPEST, Length: 522, dtype: category
Categories (4, int64): [1 < 2 < 3 < 4]

In [ ]:
pd.cut(df['BUDAPEST'], bins=[0, 10, 50, 100, 1000], labels=[1,2,3,4]).value_counts()

4    247
2    123
3    110
1     41
Name: BUDAPEST, dtype: int64

### __Challenge__

__Task__: Bin all the columns of `hungary_chickenpox`  into 4 equal sized bins.

In [ ]:
df = pd.read_csv('Datasets/hungary_chickenpox.csv')
df.head()

,Date,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,...,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,03/01/2005,168,79,30,173,169,42,136,120,162,...,130,57,2,178,66,64,11,29,87,68
1,10/01/2005,157,60,30,92,200,53,51,70,84,...,80,50,29,141,48,29,58,53,68,26
2,17/01/2005,96,44,31,86,93,30,93,84,191,...,64,46,4,157,33,33,24,18,62,44
3,24/01/2005,163,49,43,126,46,39,52,114,107,...,63,54,14,107,66,50,25,21,43,31
4,31/01/2005,122,78,53,87,103,34,95,131,172,...,61,49,11,124,63,56,7,47,85,60


In [ ]:
# Solution (error)
df.drop('Date', axis='columns').apply(lambda x: pd.qcut(x, q=4), axis='columns')

ValueError: Bin edges must be unique: array([ 0.  ,  0.  ,  2.  ,  3.25, 45.  ]).
You can drop duplicate edges by setting the 'duplicates' kwarg

Sometimess, if there are too many small values, boundary edges may get repeated, in which case pd.qcut may not work.

To handle this, set `duplicates='drop'`.

`qcut` identified duplicate edges because too many datapoints we near the same value.

In [ ]:
df.drop('Date', axis='columns').apply(lambda x: pd.qcut(x, q=4, duplicates='drop'), axis='columns')

,BUDAPEST,BARANYA,BACS,BEKES,BORSOD,CSONGRAD,FEJER,GYOR,HAJDU,HEVES,JASZ,KOMAROM,NOGRAD,PEST,SOMOGY,SZABOLCS,TOLNA,VAS,VESZPREM,ZALA
0,"(142.5, 178.0]","(73.5, 142.5]","(1.999, 40.5]","(142.5, 178.0]","(142.5, 178.0]","(40.5, 73.5]","(73.5, 142.5]","(73.5, 142.5]","(142.5, 178.0]","(1.999, 40.5]","(73.5, 142.5]","(40.5, 73.5]","(1.999, 40.5]","(142.5, 178.0]","(40.5, 73.5]","(40.5, 73.5]","(1.999, 40.5]","(1.999, 40.5]","(73.5, 142.5]","(40.5, 73.5]"
1,"(81.0, 200.0]","(55.5, 81.0]","(25.999, 43.5]","(81.0, 200.0]","(81.0, 200.0]","(43.5, 55.5]","(43.5, 55.5]","(55.5, 81.0]","(81.0, 200.0]","(25.999, 43.5]","(55.5, 81.0]","(43.5, 55.5]","(25.999, 43.5]","(81.0, 200.0]","(43.5, 55.5]","(25.999, 43.5]","(55.5, 81.0]","(43.5, 55.5]","(55.5, 81.0]","(25.999, 43.5]"
2,"(87.75, 191.0]","(32.5, 48.5]","(3.999, 32.5]","(48.5, 87.75]","(87.75, 191.0]","(3.999, 32.5]","(87.75, 191.0]","(48.5, 87.75]","(87.75, 191.0]","(48.5, 87.75]","(48.5, 87.75]","(32.5, 48.5]","(3.999, 32.5]","(87.75, 191.0]","(32.5, 48.5]","(32.5, 48.5]","(3.999, 32.5]","(3.999, 32.5]","(48.5, 87.75]","(32.5, 48.5]"
3,"(76.25, 163.0]","(41.25, 49.5]","(41.25, 49.5]","(76.25, 163.0]","(41.25, 49.5]","(13.999, 41.25]","(49.5, 76.25]","(76.25, 163.0]","(76.25, 163.0]","(41.25, 49.5]","(49.5, 76.25]","(49.5, 76.25]","(13.999, 41.25]","(76.25, 163.0]","(49.5, 76.25]","(49.5, 76.25]","(13.999, 41.25]","(13.999, 41.25]","(41.25, 49.5]","(13.999, 41.25]"
4,"(97.0, 172.0]","(62.0, 97.0]","(48.5, 62.0]","(62.0, 97.0]","(97.0, 172.0]","(6.999, 48.5]","(62.0, 97.0]","(97.0, 172.0]","(97.0, 172.0]","(6.999, 48.5]","(48.5, 62.0]","(48.5, 62.0]","(6.999, 48.5]","(97.0, 172.0]","(62.0, 97.0]","(48.5, 62.0]","(6.999, 48.5]","(6.999, 48.5]","(62.0, 97.0]","(48.5, 62.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,"(39.5, 122.0]","(9.25, 13.5]","(39.5, 122.0]","(-0.001, 9.25]","(13.5, 39.5]","(-0.001, 9.25]","(13.5, 39.5]","(13.5, 39.5]","(13.5, 39.5]","(9.25, 13.5]","(39.5, 122.0]","(-0.001, 9.25]","(9.25, 13.5]","(39.5, 122.0]","(-0.001, 9.25]","(13.5, 39.5]","(-0.001, 9.25]","(9.25, 13.5]","(39.5, 122.0]","(9.25, 13.5]"
518,"(34.5, 70.0]","(34.5, 70.0]","(24.0, 34.5]","(1.999, 16.0]","(24.0, 34.5]","(1.999, 16.0]","(1.999, 16.0]","(24.0, 34.5]","(24.0, 34.5]","(16.0, 24.0]","(24.0, 34.5]","(16.0, 24.0]","(16.0, 24.0]","(34.5, 70.0]","(34.5, 70.0]","(1.999, 16.0]","(16.0, 24.0]","(16.0, 24.0]","(34.5, 70.0]","(1.999, 16.0]"
519,"(18.0, 72.0]","(3.5, 7.0]","(7.0, 18.0]","(-0.001, 3.5]","(-0.001, 3.5]","(-0.001, 3.5]","(3.5, 7.0]","(3.5, 7.0]","(3.5, 7.0]","(-0.001, 3.5]","(18.0, 72.0]","(18.0, 72.0]","(3.5, 7.0]","(18.0, 72.0]","(3.5, 7.0]","(18.0, 72.0]","(7.0, 18.0]","(-0.001, 3.5]","(7.0, 18.0]","(7.0, 18.0]"
520,"(18.0, 83.0]","(18.0, 83.0]","(3.5, 10.5]","(-0.001, 3.5]","(10.5, 18.0]","(3.5, 10.5]","(-0.001, 3.5]","(3.5, 10.5]","(3.5, 10.5]","(10.5, 18.0]","(18.0, 83.0]","(10.5, 18.0]","(18.0, 83.0]","(10.5, 18.0]","(3.5, 10.5]","(10.5, 18.0]","(-0.001, 3.5]","(-0.001, 3.5]","(18.0, 83.0]","(-0.001, 3.5]"
